# Gridsearch

In this notebook we examine the best value for `max_depth` used in the `sklearn.ensemble.RandomForestClassifier`.

In [1]:
import datetime, joblib

import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from tqdm.auto import tqdm

from cooccurringNetwork.cooccurringNetwork import create_graph
from linkPrediction.linkPrediction import (
  determine_active_nodes, get_index, get_split, get_targets, giant_component, 
  print_status)
# from linkPrediction.linkPrediction import *

rcParams['xtick.top'] = True
rcParams['ytick.right'] = True

## Get the data

In [2]:
%%time
data        = joblib.load("temp/cleaned_data.pkl")
intentional = joblib.load('temp/intentional_cooccurrences.pkl')

CPU times: user 9.98 s, sys: 982 ms, total: 11 s
Wall time: 11 s


## Get instances

We choose not to use a lifetime and we do select only the GC to reduce class imbalance while still maintaining enough positive instances.

In [3]:
%%time
tau                         = .5
depthlimit                  = None
lifetime                    = None
stopIndex                   = int(len(intentional) * tau)
graph_tau                   = create_graph(intentional[:stopIndex])
graph_tau.graph['finalDate']= intentional[stopIndex].time
graph_tau                   = determine_active_nodes(graph_tau, 
                                                     lifetime=lifetime)
# graph_tau                 = giant_component(graph_tau)
graph                       = create_graph(intentional)
index                       = get_index(graph=graph_tau, depthlimit=depthlimit)
y                           = get_targets(graph=graph, index=index)

f, p = np.bincount(y)
print(f'From the target, {p} truck pairs will make a link.')
print(f'Class imbalance is equal to 1:{round(f/p)}.')

From the target, 6481 truck pairs will make a link.
Class imbalance is equal to 1:38749.
CPU times: user 4min 46s, sys: 15.8 s, total: 5min 2s
Wall time: 5min 31s


In [4]:
%%time
tau                         = .5
depthlimit                  = None
lifetime                    = None
stopIndex                   = int(len(intentional) * tau)
graph_tau                   = create_graph(intentional[:stopIndex])
graph_tau.graph['finalDate']= intentional[stopIndex].time
graph_tau                   = determine_active_nodes(graph_tau, 
                                                     lifetime=lifetime)
graph_tau                   = giant_component(graph_tau)
graph                       = create_graph(intentional)
index                       = get_index(graph=graph_tau, depthlimit=depthlimit)
y                           = get_targets(graph=graph, index=index)

f, p = np.bincount(y)
print(f'From the target, {p} truck pairs will make a link.')
print(f'Class imbalance is equal to 1:{round(f/p)}.')

From the target, 4517 truck pairs will make a link.
Class imbalance is equal to 1:18704.
CPU times: user 4min 52s, sys: 17.2 s, total: 5min 9s
Wall time: 5min 10s


## Get features

In [10]:
%%time
def get_node_features(
  *, graph: nx.Graph, data: pd.DataFrame, verbose: bool = False 
  ) -> dict:
  """Get the node features for the link prediction problem.

  Arguments:
  - graph: graph used to create features
  - data:  pd.DataFrame containing the columns `location`, `datetime`, 
           `entityId`.

  Returns:
  - dict of dict: First key is node feature and second key is license plate. 
  """
  if verbose: print_status("Started node feature construction.")
  final_date = max([attr['time'] for n, attr in graph.nodes(data=True)])
  trucks = [node for node in graph]
  d = data[(data['datetime'] <= final_date) & data['entity'].isin(trucks)
          ].copy()
  # d_7 and d_30 are used for the spatiotemporal features
  d_7  = d[d['datetime'] >= final_date-pd.Timedelta(7, 'D')].copy()
  d_30 = d[d['datetime'] >= final_date-pd.Timedelta(30, 'D')].copy()

  def get_driving_hours(x): return (x.dt.hour - 12).abs().mean()
  def get_weekend(x): return (x.dt.weekday >= 5).mean()
  # See paper for definitions of these node features.
  node_features = {
    "country": {node: node.split('-')[1] for node in graph},
    "ax": d.groupby("entity")["axes"].median().fillna(0).to_dict(),
    "length": d.groupby("entity")["length"].median().fillna(0).to_dict(),
    "m": d.groupby("entity")["mass"].median().fillna(0).to_dict(),
    "night": d.groupby("entity")["datetime"].agg(get_driving_hours).to_dict(),
    "weekend": d.groupby("entity")["datetime"].agg(get_weekend).to_dict(),
    "nb": {n: set(graph[n]) for n in graph},
    "vol": {n: sum([w for _, _, w in graph.edges(nbunch=n, data="weight")])
            for n in graph},
    "shortest_path": {n: dict(nx.single_source_shortest_path_length(graph, n)) 
                     for n in tqdm(graph, "shortest_path per node")},
    # Contains for every location the nuber of registrations in last
    # x days.
    "f_7":  d_7.groupby( ["entity", "location"]).size().to_dict(),
    "f_30": d_30.groupby(["entity", "location"]).size().to_dict(),
    # 365 equals all measurements.
    "f_365": d.groupby(  ["entity", "location"]).size().to_dict() 
  }

  if verbose: print_status("Ended node feature construction.")
  return node_features

def get_features(
  *, graph: nx.Graph, data: pd.DataFrame, index: list[int, int], 
  verbose: bool = True
  ) -> pd.DataFrame:
  """Returns the features for the link prediction problem.

  Args:
  - graph: Used to determine the features from.
  - data: Used to construct the features from.
  - index: List of pairs of nodes for which the features are constructed.
  """
  disable = dict(disable=not verbose)
  if verbose: print_status("Start feature construction.")

  node_features = get_node_features(graph=graph, data=data)

  if verbose: print_status("Create pd.DataFrame with index.")
  x = pd.DataFrame(index=index)

  x["truck_ax_sum"] = [
    node_features["ax"][u] + node_features["ax"][v] 
    for u, v in tqdm(index, desc="truck_ax_sum", disable=not verbose)]
  x["truck_ax_diff"] = [
    abs(node_features["ax"][u] - node_features["ax"][v]) 
    for u, v in tqdm(index, desc="truck_ax_diff", disable=not verbose)]          
  x["truck_len_sum"] = [
    node_features["length"][u] + node_features["length"][v] 
    for u, v in tqdm(index, desc="truck_len_sum", disable=not verbose)]
  x["truck_len_diff"] = [
    abs(node_features["length"][u] - node_features["length"][v]) 
    for u, v in tqdm(index, desc="truck_len_diff", disable=not verbose)]
  x["truck_mass_sum"] = [
    node_features["m"][u] + node_features["m"][v] 
    for u, v in tqdm(index, desc="truck_mass_sum", disable=not verbose)]
  x["truck_mass_diff"] = [
    abs(node_features["m"][u] - node_features["m"][v]) 
    for u, v in tqdm(index, desc="truck_mass_diff", disable=not verbose)]
  x["night_sum"] = [
    node_features["night"][u] + node_features["night"][v] 
    for u, v in tqdm(index, desc="night_sum", disable=not verbose)]
  x["night_diff"] = [
    abs(node_features["night"][u] - node_features["night"][v]) 
    for u, v in tqdm(index, desc="night_diff", disable=not verbose)]
  x["weekend_sum"] = [
    node_features["weekend"][u] + node_features["weekend"][v]
    for u, v in tqdm(index, desc="weekend_sum", disable=not verbose)]
  x["weekend_diff"] = [
    abs(node_features["weekend"][u] - node_features["weekend"][v]) 
    for u, v in tqdm(index, desc="weekend_diff", disable=not verbose)]
  x["degree_sum"] = [
    len(node_features["nb"][u]) + len(node_features["nb"][v]) 
    for u, v in tqdm(index, desc="degree_sum", disable=not verbose)]
  x["degree_diff"] = [
    abs(len(node_features["nb"][u]) - len(node_features["nb"][v]))
    for u, v in tqdm(index, desc="degree_diff", disable=not verbose)]
  x["volume_sum"] = [
    node_features["vol"][u] + node_features["vol"][v] 
    for u, v in tqdm(index, desc="volume_sum", disable=not verbose)]
  x["volume_diff"] = [
    abs(node_features["vol"][u] - node_features["vol"][v])
    for u, v in tqdm(index, desc="volume_diff", disable=not verbose)]       
  x["country_same"] = [
    node_features["country"][u] == node_features["country"][v]
    for u, v in tqdm(index, desc="country_same", disable=not verbose)]
  x["nb_total"] = [
    len(node_features["nb"][u] | node_features["nb"][v])
    for u, v in tqdm(index, desc="nb_total", disable=not verbose)]
  x["nb_common"] = [
    len(node_features["nb"][u] & node_features["nb"][v])
    for u, v in tqdm(index, desc="nb_common", disable=not verbose)]
  x["shortest_path"] = [
    node_features["shortest_path"][u].get(v, 100)
    for u, v in tqdm(index, desc="shortest_path", disable=not verbose)]

  locations = {loc for _, loc in node_features["f_365"].keys()}
  for location in tqdm(locations, desc="spatiotemporal features", 
                       disable=not verbose):
    x[f'f_7_{location}'] = [node_features["f_7"].get((u, location), 0) + 
                            node_features["f_7"].get((v, location), 0)
                            for u, v in index]
    x[f'f_30_{location}'] = [node_features["f_30"].get((u, location), 0) + 
                             node_features["f_30"].get((v, location), 0)
                             for u, v in index]
    x[f'f_365_{location}'] = [node_features["f_365"].get((u, location), 0) + 
                              node_features["f_365"].get((v, location), 0)
                              for u, v in index]

  return x.loc[:, x.any()] # Remove columns with only zeros.

x = get_features(graph=graph_tau, data=data, index=index)

18:29:23 Start feature construction.


shortest_path per node:   0%|          | 0/13001 [00:00<?, ?it/s]

18:46:20 Create pd.DataFrame with index.


truck_ax_sum:   0%|          | 0/84488577 [00:00<?, ?it/s]

truck_ax_diff:   0%|          | 0/84488577 [00:00<?, ?it/s]

...

nb_total:   0%|          | 0/84488577 [00:00<?, ?it/s]

nb_common:   0%|          | 0/84488577 [00:00<?, ?it/s]

shortest_path:   0%|          | 0/84488577 [00:00<?, ?it/s]

spatiotemporal features:   0%|          | 0/6 [00:00<?, ?it/s]

CPU times: user 1h 16min 12s, sys: 5min 32s, total: 1h 21min 44s
Wall time: 1h 21min 46s


## Split in train and test
Create a train and test split. The Split object contains attributes `xtrain`, `xtest`, `ytrain` and `ytest`.

In [ ]:
%%time
split = get_split(x, y)
joblib.dump(split, "temp/split.pkl")

## Perform gridsearch

In [ ]:
split = joblib.load('temp/split.pkl')

In [4]:
%%time
randomforest = RandomForestClassifier(
  n_estimators=128, random_state=1, class_weight="balanced", n_jobs=128)

clf = GridSearchCV(
  randomforest, {"max_depth": [1, 2, 3, 4, 8, 16, 32, 64, None]},
  scoring=["average_precision", "roc_auc"], n_jobs=2, refit=False, verbose=10, 
  return_train_score=True)

clf.fit(split.xtrain, split.ytrain)
joblib.dump(clf, 'temp/gridsearch-clf.pkl')

Fitting 5 folds for each of 9 candidates, totalling 45 fits


exception calling callback for <Future at 0x7f6b14334760 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/home/bruingjde/.conda/envs/truckCodriving/lib/python3.9/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/bruingjde/.conda/envs/truckCodriving/lib/python3.9/site-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/home/bruingjde/.conda/envs/truckCodriving/lib/python3.9/site-packages/joblib/parallel.py", line 792, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/home/bruingjde/.conda/envs/truckCodriving/lib/python3.9/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/bruingjde/.conda/envs/truckCodriving/lib/python3.9/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/home/bruing

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

## ROC AUC

In [5]:
%%time
for label, score in {'Train': "mean_train_roc_auc", 
                     "Validation": "mean_test_roc_auc"}.items():
  pd.Series(clf.cv_results_[score], 
            index=[param['max_depth'] for param in clf.cv_results_['params']]
           ).loc[[1, 2, 3, 4, 8, 16, 32, 64, None]].plot(label=label)
plt.legend()
plt.xlabel('max_depth')
plt.ylabel('ROC AUC')

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

## Average precision

In [6]:
%%time
for label, score in {'Train': "mean_train_average_precision",
                     "Validation": "mean_test_average_precision"}.items():
  pd.Series(clf.cv_results_[score], 
            index=[param['max_depth'] for param in clf.cv_results_['params']]
           ).loc[[1, 2, 3, 4, 8, 16, 32, 64, None]].plot(label=label)
plt.legend()
plt.yscale("log")
plt.xlabel('max_depth')
plt.ylabel('Average precision')

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

## Select best hyperparameters
We choose a value of `max_depth=3`. Because of the higher performance while still not (severely) overfitting.